# Calculating the Implied Earnings Move Using Options Prices

Earnings day can be a pivotal moment for a company's share price. The confluence of expectations and reality is a tradable event, drawing crowds to the options market. Observing the surrounding action can provide insight into the consensus view on, and general sentiment of, the company.

The cost of a straddle - the combined price of an at-the-money call and put - is a common way to gauge the near-term volatility. It's the market's expectation of the price band until expiration. While this includes time value, the isolated price of volatility will generally be higher for the expiry immediately following an earnings release.

Have a look at companies that trade weekly options and are reporting a on Thursday. If they report after the close, the price of the one-day straddle at the bell will be the purest sample of information.

The cells below will demonstrate how to get the data from free sources, using the OpenBB Platform.

In [ ]:
# If using in Google Colab, install the OpenBB library.

!pip install openbb["all"]

# Restart the runtime before the next block

In [1]:
from datetime import datetime, timedelta

import pandas as pd
from openbb import obb

obb.user.preferences.output_type = "dataframe"
obb.user.credentials.nasdaq_api_key = "PLACE_HOLDER" # You don't actually need to replace this.

If the earnings date falls on an option expiry, contracts expiring that day will not provide exposure to the after-market earnings reports.

In [2]:
# Lookup some upcoming earnings dates and sort them by market cap.

earnings_calendar = obb.equity.calendar.earnings(
    start_date=(datetime.now()+timedelta(days=1)).date(),
    end_date = (datetime.now()+timedelta(days=14)).date(),
    provider="nasdaq"
)

earnings_calendar.sort_values(by=["market_cap", "num_estimates"], ascending=False).head(20)

,report_date,symbol,name,eps_previous,eps_consensus,num_estimates,period_ending,previous_report_date,reporting_time,market_cap
26,2024-03-07,AVGO,Broadcom Inc.,9.83,9.06,8.0,2024-01,2023-03-02,after-hours,6.108766e+11
27,2024-03-07,COST,Costco Wholesale Corporation,3.30,3.60,14.0,2024-02,2023-03-02,after-hours,3.259489e+11
553,2024-02-28,CRM,"Salesforce, Inc.",1.01,1.72,16.0,2024-01,2023-03-01,after-hours,2.842532e+11
554,2024-02-28,RY,Royal Bank Of Canada,2.26,2.10,4.0,2024-01,2023-03-01,pre-market,1.380355e+11
796,2024-02-27,LOW,"Lowe's Companies, Inc.",2.28,1.68,15.0,2024-01,2023-03-01,pre-market,1.325692e+11
28,2024-03-07,PBR,Petroleo Brasileiro S.A.- Petrobras,1.25,1.13,3.0,2023-12,2023-03-02,not-supplied,1.146585e+11
555,2024-02-28,TJX,"TJX Companies, Inc. (The)",0.89,1.11,10.0,2024-01,2023-02-22,pre-market,1.129534e+11
29,2024-03-07,PBR.A,Petroleo Brasileiro S.A.- Petrobras,1.25,NaN,5.0,2023-12,None,not-supplied,1.108105e+11
383,2024-02-29,BUD,Anheuser-Busch Inbev SA,0.98,0.76,4.0,2023-12,2023-03-02,pre-market,1.101379e+11
384,2024-02-29,TD,Toronto Dominion Bank (The),1.64,1.46,4.0,2024-01,2023-03-02,pre-market,1.074778e+11


In [3]:
# Get the last price of the underlying stock.
symbol = "CRM"

quote = obb.equity.price.quote(symbol, provider="yfinance").T

last_price = quote.loc["last_price", 0]

print(f"Last Price: ${last_price}")


Last Price: $292.8


In [11]:
# Get the options chains data.

options = obb.derivatives.options.chains(symbol, provider="cboe")

print(options.expiration.unique()[0])

2024-03-01


In [12]:
# Filter for the earnings expiration.
expiration = datetime(2024,3,1).date() # This date will not be evergreen, so change it to suit.
chain = options.query("`expiration` == @expiration")

chain

,contract_symbol,expiration,strike,option_type,open_interest,volume,theoretical_price,last_trade_price,tick,bid,...,change,change_percent,implied_volatility,delta,gamma,theta,vega,rho,last_trade_timestamp,dte
0,CRM240301C00135000,2024-03-01,135.0,call,0,0,157.9118,0.00,no_change,157.15,...,0.000,0.000,2.1031,0.9997,0.0000,0.0000,0.0004,0.0210,NaT,5
1,CRM240301P00135000,2024-03-01,135.0,put,64,5,0.0038,0.01,down,0.00,...,-0.070,-0.875,1.6854,-0.0002,0.0000,-0.0026,0.0003,0.0000,2024-02-23 13:27:03,5
2,CRM240301C00140000,2024-03-01,140.0,call,0,0,152.9175,0.00,no_change,152.00,...,0.000,0.000,0.0000,0.9997,0.0000,0.0000,0.0004,0.0221,NaT,5
3,CRM240301P00140000,2024-03-01,140.0,put,0,0,0.0040,0.00,no_change,0.00,...,0.000,0.000,1.6954,-0.0003,0.0000,-0.0028,0.0004,0.0000,NaT,5
4,CRM240301C00145000,2024-03-01,145.0,call,0,0,147.9232,0.00,no_change,147.10,...,0.000,0.000,1.7827,0.9997,0.0000,0.0000,0.0004,0.0232,NaT,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,CRM240301P00377500,2024-03-01,377.5,put,0,0,84.7275,0.00,no_change,83.80,...,0.000,0.000,0.0000,-0.9971,0.0009,-0.0457,0.0032,-0.0049,NaT,5
164,CRM240301C00380000,2024-03-01,380.0,call,41,3,0.0825,0.14,up,0.08,...,0.005,0.037,0.8024,0.0087,0.0008,-0.0416,0.0096,0.0004,2024-02-23 14:41:10,5
165,CRM240301P00380000,2024-03-01,380.0,put,0,0,87.2249,0.00,no_change,86.30,...,0.000,0.000,0.0000,-0.9982,0.0006,-0.0432,0.0014,0.0000,NaT,5
166,CRM240301C00390000,2024-03-01,390.0,call,51,59,0.0441,0.11,up,0.04,...,0.010,0.100,0.8468,0.0049,0.0004,-0.0242,0.0057,0.0002,2024-02-23 13:57:21,5


In [6]:
# Find the nearest strike price to the last price.

strikes = chain.strike.to_frame()

call_strike = strikes.loc[strikes.query("`strike` > @last_price").idxmin()]["strike"].iloc[0]
put_strike = strikes.loc[strikes.query("`strike` < @last_price").idxmax()]["strike"].iloc[0]

print(f"Last Price: ${last_price}\n\nNearest Call Strike: ${call_strike}\n\nNearest Put Strike: ${put_strike}")

Last Price: $292.8

Nearest Call Strike: $295.0

Nearest Put Strike: $292.5


In [13]:
# Using the same strike for the put as the call.

atm_call = chain.query("`strike` == @call_strike and `option_type` == 'call'")
atm_put = chain.query("`strike` == @call_strike and `option_type` == 'put'")

atm = pd.concat([atm_call, atm_put])
straddle_price = round(atm.ask.sum(), 2)
upper_price = round((last_price*(1+(straddle_price/last_price))), 2)
lower_price = round((last_price*(1-(straddle_price/last_price))), 2)

# Calculate the expected daily move
days = (atm.expiration.iloc[0] - datetime.now().date()).days
implied_move = ((1+ straddle_price/last_price)**(1/days) - 1) * 100

print(
    f"Cost of Straddle: ${straddle_price}"
    f"\nCost as a % of Share Price: {round((straddle_price/last_price) * 100, 4)}%"
    f"\nUpper Breakeven Price: ${upper_price}"
    f"\nLower Breakeven Price: ${lower_price}"
    f"\nImplied Daily Move: {round(implied_move, 4)}%\n"
)

atm

Cost of Straddle: $21.95
Cost as a % of Share Price: 7.4966%
Upper Breakeven Price: $314.75
Lower Breakeven Price: $270.85
Implied Daily Move: 1.4563%



,contract_symbol,expiration,strike,option_type,open_interest,volume,theoretical_price,last_trade_price,tick,bid,...,change,change_percent,implied_volatility,delta,gamma,theta,vega,rho,last_trade_timestamp,dte
96,CRM240301C00295000,2024-03-01,295.0,call,540,315,9.8163,10.05,down,9.85,...,-0.375,-0.0360,0.6666,0.4904,0.0148,-0.7900,0.1620,0.0254,2024-02-23 15:59:01,5
97,CRM240301P00295000,2024-03-01,295.0,put,141,47,11.7541,11.85,up,11.75,...,0.375,0.0327,0.6658,-0.5107,0.0148,-0.7925,0.1619,-0.0272,2024-02-23 15:59:59,5


In [14]:
# Using the same strike for the call as the put.

atm_call = chain.query("`strike` == @put_strike and `option_type` == 'call'")
atm_put = chain.query("`strike` == @put_strike and `option_type` == 'put'")

atm = pd.concat([atm_call, atm_put])
straddle_price = round(atm.ask.sum(), 2)
upper_price = round((last_price*(1+(straddle_price/last_price))), 2)
lower_price = round((last_price*(1-(straddle_price/last_price))), 2)

# Calculate the expected daily move
days = (atm.expiration.iloc[0] - datetime.now().date()).days
implied_move = ((1+ straddle_price/last_price)**(1/days) - 1) * 100

print(
    f"Cost of Straddle: ${straddle_price}"
    f"\nCost as a % of Share Price: {round((straddle_price/last_price) * 100, 4)}%"
    f"\nUpper Breakeven Price: ${upper_price}"
    f"\nLower Breakeven Price: ${lower_price}"
    f"\nImplied Daily Move: {round(implied_move, 4)}%\n"
)

atm

Cost of Straddle: $21.8
Cost as a % of Share Price: 7.4454%
Upper Breakeven Price: $314.6
Lower Breakeven Price: $271.0
Implied Daily Move: 1.4466%



,contract_symbol,expiration,strike,option_type,open_interest,volume,theoretical_price,last_trade_price,tick,bid,...,change,change_percent,implied_volatility,delta,gamma,theta,vega,rho,last_trade_timestamp,dte
94,CRM240301C00292500,2024-03-01,292.5,call,155,154,10.9659,11.25,up,11.0,...,-0.350,-0.0302,0.6679,0.5271,0.0148,-0.7864,0.1617,0.0272,2024-02-23 15:50:02,5
95,CRM240301P00292500,2024-03-01,292.5,put,96,119,10.4037,10.55,up,10.4,...,0.375,0.0369,0.6640,-0.4739,0.0148,-0.7890,0.1616,-0.0254,2024-02-23 15:59:59,5


In [10]:
# When using the same strike, one option will end up being in-the-money.
# This may inflate the cost of the position.
# It can be more favourable to use the nearest OTM strike to each instead.

atm_call = chain.query("`strike` == @call_strike and `option_type` == 'call'")
atm_put = chain.query("`strike` == @put_strike and `option_type` == 'put'")

atm = pd.concat([atm_call, atm_put])
straddle_price = round(atm.ask.sum(), 2)
upper_price = round((last_price*(1+(straddle_price/last_price))), 2)
lower_price = round((last_price*(1-(straddle_price/last_price))), 2)

# Calculate the expected daily move
days = (atm.expiration.iloc[0] - datetime.now().date()).days
implied_move = ((1+ straddle_price/last_price)**(1/days) - 1) * 100

print(
    f"Cost of Straddle: ${straddle_price}"
    f"\nCost as a % of Share Price: {round((straddle_price/last_price) * 100, 4)}%"
    f"\nUpper Breakeven Price: ${upper_price}"
    f"\nLower Breakeven Price: ${lower_price}"
    f"\nImplied Daily Move: {round(implied_move, 4)}%\n"
)

atm

Cost of Straddle: $20.55
Cost as a % of Share Price: 7.0184%
Upper Breakeven Price: $313.35
Lower Breakeven Price: $272.25
Implied Daily Move: 1.3659%



,contract_symbol,expiration,strike,option_type,open_interest,volume,theoretical_price,last_trade_price,tick,bid,...,change,change_percent,implied_volatility,delta,gamma,theta,vega,rho,last_trade_timestamp,dte
96,CRM240301C00295000,2024-03-01,295.0,call,540,315,9.8163,10.05,down,9.85,...,-0.375,-0.0360,0.6666,0.4904,0.0148,-0.790,0.1620,0.0254,2024-02-23 15:59:01,5
95,CRM240301P00292500,2024-03-01,292.5,put,96,119,10.4037,10.55,up,10.40,...,0.375,0.0369,0.6640,-0.4739,0.0148,-0.789,0.1616,-0.0254,2024-02-23 15:59:59,5
